# IMPORTING LIBRARIES AND DATASET

In [1]:
import pandas as pd 

In [2]:
data = pd.read_csv('train.csv')

In [3]:
data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
data.shape 

(20800, 5)

In [5]:
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
# Dropping the NAN values
data = data.dropna()

In [7]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [8]:
# Getting the X split or independent variable from the dataset
X = data.drop('label',axis=1)

In [9]:
# Getting the Y split or the dependent variable
Y = data['label']

In [10]:
X.shape

(18285, 4)

In [11]:
Y.shape

(18285,)

In [12]:
import tensorflow as tf

In [13]:
print(tf.__version__)

2.10.0


In [14]:
from tensorflow.keras.layers import Embedding # word to vec
from tensorflow.keras.preprocessing.sequence import pad_sequences # making size of sentences same
from tensorflow.keras.models import Sequential # sequential layer 
from tensorflow.keras.preprocessing.text import one_hot # to encode categorical features
from tensorflow.keras.layers import LSTM 
from tensorflow.keras.layers import Dense # used in the case of deep neural network

In [15]:
# vocabulary size
voc_size = 5000

# ONE HOT REPRESENTATION

In [16]:
messages = X.copy()

In [17]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
messages

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [19]:
messages.reset_index(inplace = True)

In [20]:
messages

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...,...
18280,20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [21]:
import nltk
import re 
from nltk.corpus import stopwords

In [23]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sid80\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [24]:
# Data Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [25]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [26]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [27]:
onehot_repr = [one_hot(words,voc_size)for words in corpus]
onehot_repr

[[209, 4152, 3555, 3618, 1011, 558, 830, 444, 871, 3204],
 [1072, 3897, 4515, 527, 366, 682, 662],
 [1360, 34, 4358, 3045],
 [550, 1814, 3186, 3558, 1771, 2482],
 [4772, 366, 254, 3007, 1505, 1204, 366, 509, 2870, 3527],
 [1342,
  814,
  3100,
  2816,
  1600,
  4142,
  3467,
  4799,
  2204,
  2128,
  2339,
  1539,
  1482,
  1836,
  662],
 [537, 1022, 2043, 1586, 3514, 397, 549, 673, 3508, 189, 1623],
 [1858, 2660, 4728, 1250, 2612, 1988, 4142, 4885, 3508, 189, 1623],
 [3752, 2613, 191, 2972, 1225, 3845, 1874, 3754, 4142, 471],
 [1295, 1241, 1588, 3639, 3067, 2454, 4836, 2010],
 [4664, 3350, 4554, 1752, 19, 3685, 1924, 1250, 222, 3664, 617],
 [3558, 194, 1011, 3845, 4142, 2612],
 [2220, 716, 4752, 2767, 4032, 2969, 2612, 696, 2471],
 [3107, 4340, 4815, 1234, 4577, 138, 3015, 3508, 189, 1623],
 [103, 4892, 889, 2553, 3344, 3508, 189, 1623],
 [185, 591, 4065, 713, 2021, 3128, 564, 4077, 4648, 337],
 [1446, 681, 3897],
 [1828, 3421, 78, 80, 4142, 1989, 474, 662],
 [2315, 4430, 4515, 3885, 

In [28]:
onehot_repr[1]

[1072, 3897, 4515, 527, 366, 682, 662]

# EMBEDDING REPRESENTATION

In [29]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[ 209 4152 3555 ...    0    0    0]
 [1072 3897 4515 ...    0    0    0]
 [1360   34 4358 ...    0    0    0]
 ...
 [1312  369 4662 ...    0    0    0]
 [1475 2612 1788 ...    0    0    0]
 [1950 3026 4213 ...    0    0    0]]


In [30]:
embedded_docs[1]

array([1072, 3897, 4515,  527,  366,  682,  662,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [31]:
embedded_docs[0]

array([ 209, 4152, 3555, 3618, 1011,  558,  830,  444,  871, 3204,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0])

In [32]:
# CREATING MODEL
embedding_vector_features = 40 ##feature representation
# it means that every index of it is converted into the 40 features
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
len(embedded_docs),Y.shape

(18285, (18285,))

In [35]:
# now we have converted the words into vectors
# sentence -> messages -> corpus -> one_hot encode -> embedded docs (padding) -> X-final
import numpy as np
X_final=np.array(embedded_docs)
Y_final=np.array(Y)

In [36]:
X_final.shape , Y_final.shape

((18285, 20), (18285,))

In [38]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y_final, test_size=0.33, random_state=42)

# MODEL TRAINING 

In [39]:
# final training
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 30ms/step - loss: 0.3183 - accuracy: 0.8425 - val_loss: 0.1967 - val_accuracy: 0.9167
Epoch 2/10
192/192 [==============================] - 5s 24ms/step - loss: 0.1392 - accuracy: 0.9459 - val_loss: 0.1968 - val_accuracy: 0.9183
Epoch 3/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0930 - accuracy: 0.9654 - val_loss: 0.2792 - val_accuracy: 0.9097
Epoch 4/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0662 - accuracy: 0.9791 - val_loss: 0.2993 - val_accuracy: 0.9062
Epoch 5/10
192/192 [==============================] - 5s 27ms/step - loss: 0.0507 - accuracy: 0.9820 - val_loss: 0.3145 - val_accuracy: 0.9109
Epoch 6/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0350 - accuracy: 0.9886 - val_loss: 0.3600 - val_accuracy: 0.9097
Epoch 7/10
192/192 [==============================] - 5s 25ms/step - loss: 0.0264 - accuracy: 0.9915 - val_loss: 0.4546 - val_accuracy: 0.9064

In [41]:
# dropout is used for dropping out of the node(neurons) for the regularization and reduce the overfitting problem

# from tensorflow.keras.layers import Dropout
## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


# PERFORMANCE METRICS AND ACCURACY

In [42]:
Y_pred = model.predict(X_test)

189/189 [==============================] - 2s 6ms/step


In [44]:
Y_pred=np.where(Y_pred > 0.6, 1,0) ##AUC ROC Curve

In [45]:
from sklearn.metrics import confusion_matrix

In [46]:
confusion_matrix(Y_test,Y_pred)

array([[3103,  316],
       [ 236, 2380]], dtype=int64)

In [48]:
from sklearn.metrics import accuracy_score
accuracy_score(Y_test,Y_pred)

0.9085335542667772

In [50]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.88      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

